# CS5481 - Tutorial 9
## Information Retrieval

Welcome to CS5481 tutorial 9. In this tutorial, you will learn how to use classic information retrieval methods in practice.

Information retrieval in data science refers to the process of retrieving relevant information or data from a collection or database to satisfy a user's query. It involves techniques and methods for searching, retrieving, and presenting information in a way that is meaningful and useful to users.

## preparation
- Python
- Python Libraries
  - numpy

# Context
1. Boolean Retrieval

2. Vector Space Model
3. BM25 Model

# Boolean Retrieval


Boolean retrieval is a classic information retrieval technique that allows users to retrieve relevant documents or data by using Boolean operators and logical expressions.

In Boolean retrieval, a document or data item is represented as a set of terms or keywords. Users can construct queries by combining these terms using Boolean operators such as AND, OR, and NOT. The operators enable users to specify the relationships between terms and refine their search criteria.

The basic Boolean operators are:

*   AND: Retrieves documents or data items that contain all the specified terms.
*   OR: Retrieves documents or data items that contain at least one of the specified terms.
*   NOT: Excludes documents or data items that contain the specified term.

By combining these operators, users can create complex Boolean expressions to precisely define their search requirements. The result of a Boolean retrieval is a set of documents or data items that match the specified criteria.

Let's consider the following documents:

In [1]:
docs = ["The top surface of the Model A's car-like exterior is a mesh so that air can pass through to eight propellers inside the body which provide lift.",
       "But flying any distance using these alone, without the assistance of wings, would require prohibitive amounts of power.",
       "Alef's proposed solution is novel - for longer flights the Model A transforms into a biplane.",
       "It's an ingenious idea, but is it a practical one?",
       "The mesh, as visualised, might also cause significant aerodynamic drag, he adds."]

Q1: construct a vocabulary table for these documents

In [2]:
import numpy as np
vocabs = [v.strip("-").strip(".").strip("?").strip(",") for doc in docs for v in doc.strip().split()]
# remove repeated vocabs
vocabs = list(set(vocabs))
vocabs

ModuleNotFoundError: No module named 'numpy'

The given code snippet uses list comprehension to extract individual words from a nested list of documents (docs). It removes common punctuation marks from each word and creates a list of unique vocabulary words (vocabs) by converting it to a set and then back to a list.

Q2. Represent documents with One-Hot vectors

In [ ]:
# Recap: What is a one-hot vector?
vocab = ["hello", "how", "are", "you", "world"]
doc1 = "hello world"
doc2 = "how are you"
v_doc1 = [1,0,0,0,1]
v_doc2 = [0,1,1,1,0]

In [ ]:
one_hot_docs = np.zeros((len(docs), len(vocabs)))
print(one_hot_docs.shape) # doc_num x vocab_len
for i, doc in enumerate(docs):
    for v in doc.strip().split():
        v = v.strip("-").strip(".").strip("?").strip(",")
        one_hot_docs[i][vocabs.index(v)] = 1
one_hot_docs

The given code snippet creates a matrix one_hot_docs of size (number of documents, size of vocabulary), initialized with zeros. It then iterates over each document in the docs list and each word in the document. It removes common punctuation marks from the word, finds its index in the vocabs list, and sets the corresponding entry in the one_hot_docs matrix to 1. Finally, it returns the resulting one-hot encoded matrix one_hot_docs.

Q3. Retrieval docs satisfying the following requirements with Boolean Model
1. Model OR power
2. Model AND air

1. We create a one-hot encoded vector **retri_1** where the entries corresponding to the indices of "Model" and "power" are set to 1. Then, it performs **element-wise multiplication** between the one-hot encoded documents (one_hot_docs) and the retrieval vector (retri_1), followed by **summing** the values along the rows. The resulting retri_1_results contains the sum of occurrences of "Model" or "power" in each document. Finally, it prints the documents where **the sum is greater than or equal to 1**, indicating that the document satisfies the Boolean condition of having either "Model" **OR** "power" present.

In [ ]:
Model_id = vocabs.index("Model")
power_id = vocabs.index("power")

retri_1 = np.zeros(len(vocabs))
retri_1[Model_id] = 1
retri_1[power_id] = 1

retri_1_results = np.sum(one_hot_docs * retri_1, axis=1)
print("Retrieved Docs for \"Model OR power\": ")
for index, doc in enumerate(docs):
  if retri_1_results[index] >= 1:
    print(index, docs[index])


2. We create a one-hot encoded vector retri_2 where the entries corresponding to the indices of "Model" and "air" are set to 1. Then, it performs element-wise multiplication between the one-hot encoded documents (one_hot_docs) and the retrieval vector (retri_2), followed by summing the values along the rows. The resulting retri_2_results contains the sum of occurrences of both "Model" and "air" in each document. Finally, it prints the documents where the sum is greater than or equal to 2, indicating that the document satisfies the Boolean condition of having both "Model" and "air" present.



In [ ]:
Model_id = vocabs.index("Model")
air_id = vocabs.index("air")

retri_2 = np.zeros(len(vocabs))
retri_2[Model_id] = 1
retri_2[air_id] = 1

retri_2_results = np.sum(one_hot_docs * retri_2, axis=1)
print("\nRetrievaled Docs for \"Model AND air\": ")
for index, doc in enumerate(docs):
  if retri_2_results[index] >= 2:
    print(index, docs[index])

# Vector Space Model

Here, we mainly use TF-IDF method to retrieve documents

TF-IDF is calculated by multiplying two components: term frequency (TF) and inverse document frequency (IDF). TF measures the frequency of a term within a document, indicating its importance within that specific document. IDF, on the other hand, quantifies the rarity or uniqueness of a term across the entire document collection. It assigns higher weights to terms that appear less frequently in the collection, as they are considered more informative.

The TF-IDF score for a term in a document is obtained by multiplying the term's frequency (TF) in the document by the inverse document frequency (IDF) of the term across the corpus. The higher the TF-IDF score, the more significant the term is to the document. This method helps in identifying important and relevant terms for document retrieval, as well as ranking documents based on their relevance to a query.

Q1. Represent documents with TF-IDF Model

In [ ]:
import numpy as np

corpus = [
    'this is the first document',
    'this is the second document',
    'and the third one',
    'is this the first document'
]

# tokenize words
word_list = []
for i in range(len(corpus)):
    word_list.append(corpus[i].split(' '))
word_list

In [ ]:
# assign an id to each word and obtain each word's frequency
from collections import Counter
dictionary = Counter([v for item in word_list for v in item])
dictionary

In [ ]:
# represent each document with word frequency in current documents
words = list(dictionary.keys())
print(words)
tf = np.zeros((len(word_list), len(dictionary)))
for i, doc in enumerate(word_list):
    for v in doc:
        # word freq in current document
        tf[i, words.index(v)] += 1 / len(doc)
tf

In [ ]:
# represent each document with word inverse document freqency
print(words)
idf = np.zeros((len(word_list), len(dictionary)))
for i, doc in enumerate(word_list):
    for v in doc:
        idf[i, words.index(v)] = 1

# The number of documents containing the current word
idf = idf.sum(0)+1
idf = np.log(len(word_list) / idf)
idf

Certainly, the IDF (inverse document frequency) calculation formula may vary slightly in different contexts. For example, in some cases, a constant value, such as 1, is added to the denominator to prevent division by zero. Additionally, there are smoothing techniques where both the numerator and denominator are incremented by 1. These variations in the formula are used to handle specific scenarios and prevent mathematical issues, ensuring the IDF calculation is robust and effective.

Here, we add 1 to the denominator to avoid division by zero (i.e., when the word is not present in any document). If a word is more common, its denominator will be larger, resulting in a smaller and closer-to-zero inverse document frequency. Then we obtained the result through log operation.

In [ ]:
tfidf = tf * idf
tfidf

Q2. Compute similarity between the following document and the above documents based on tfidf with dot product

In [ ]:
query_doc = "this is second document"
tf_query = np.zeros(len(dictionary))
for v in query_doc.split():
    tf_query[words.index(v)] += 1 / len(query_doc.split())
print("tf_query: ", tf_query)
idf_query = np.zeros(len(dictionary))
for v in query_doc.split():
    idf_query[words.index(v)] = idf[words.index(v)]
print("idf_query: ", idf_query)
tfidf_query = tf_query * idf_query
print("tfidf_query: ", tfidf_query)

Once we have the TF-IDF vectors representing the documents, we can use them for document retrieval by vectorizing the search query and calculating the distances between the search vector and the document vectors. This allows us to determine which documents are closer or more similar to the search vector.

To do this, we vectorize the search query using the same TF-IDF representation as the documents and then calculate the distance between the search vector and each document vector. For cosine similarity, we compute the dot product between the search vector and each document vector.

In [ ]:
# compute cosine similary
similarity = tfidf_query * tfidf
similarity.sum(1)

Q3. Why is the IDF (inverse document frequency) in the TF-IDF algorithm calculated using a logarithm?

A3.1 First of all, for those terms with particularly high term frequencies that appear in almost every document (such as "the" and "is"), the number of documents in the collection containing these terms is approximately equal to the total number of documents, i.e., N/n = 1 (where N/n is always greater than 1). These terms would have a high weight, even though they lack discriminative power, which is not in line with our expectations. By using IDF (with the logarithm function), when log(1) = 0, the weight of these terms calculated by TF-IDF becomes 0, which aligns with our expectations.

A3.2 Another reason is that using logarithm can prevent weight explosion. If certain words appear in only one or a few documents (such as typographical errors), without logarithm, the IDF would be very large (due to a very small denominator), which would impact their weights. However, using logarithm can mitigate this effect.

# BM25 Model

The BM25 (Best Matching 25) model is a ranking function commonly used in information retrieval and search engines. It is an improvement over the earlier TF-IDF model. BM25 takes into account factors such as term frequency, document length, and document frequency to calculate a relevance score between a search query and a document.

$Score(Q, d) = \sum_{i}^{n}W_i R(q_i, d)$, where $Q$ is the query, $d$ is a document, $n$ is the number of words in $Q$, $q_i$ is the $i$-th word in query. $W_i$ is the weight of this word, $R(q_i, d)$ is the relevant score bewteen word $q_i$ and document $d$.

In BM2.5 Model,
$W_i = log\frac{N-df_i+0.5}{df_i+0.5}$, where $N$ is the numbe of all documents in document database, $df_i$ is the number of documents containing word $q_i$.
Based on the influence of IDF, the greater the number of documents that contain a term $q_i$, the less significant or distinctive is $q_i$. A smaller IDF value signifies a lower level of importance or discriminative ability for $q_i$. Hence, IDF can be utilized to characterize the similarity between $q_i$ and documents.

$R(q_i, d) = \frac{f_i(k_1+1)}{f+K} * \frac{qf_i(k_2+1)}{qf_i+k_2}$,
$K = k_1 * (1 -b + b * \frac{dl}{avg\_dl})$, where $k_1$, $k_2$, $b$ are tunable parameters of the BM25 model. $k_1$ controls the impact of term frequency in the document on the relevance score. $k_2$ determines the effect of term frequency in the query. $b$ controls the impact of document length normalization.
$dl$ is the length of docuemnt $d$, $avg\_dl$ is the average length of all documents.
Longer documents tend to have higher term frequencies simply because they contain more words, which can potentially lead to a bias in relevance ranking.
By incorporating document length in the BM25 model, the impact of document length on the relevance score is normalized.

The first term, $\frac{f_i(k_1+1)}{f+K}$, represents the impact of term frequency in the document, where $f_i$ is the number word $q_i$ appearing in all documents, $f$ is the total term frequency in the document.
The second term, $\frac{qf_i(k_2+1)}{qf_i+k_2}$, represents the impact of term frequency in the query. $qf_i$ is the number word $q_i$ appearing in query.

Q1. Construct a BM25 Model

In [ ]:
import numpy as np
from collections import Counter


class BM25_Model(object):
    def __init__(self, documents_list, k1=2, k2=1, b=0.5):
        # documents and each document is a list of words
        self.documents_list = documents_list
        self.documents_number = len(documents_list)
        # average document length
        self.avg_documents_len = sum([len(document) for document in documents_list]) / self.documents_number
        # save each word's frequency in each document
        self.f = []
        # word's weight
        self.idf = {}
        self.k1 = k1
        self.k2 = k2
        self.b = b
        # obtain f and idf from input documents
        self.init()

    def init(self):
        #
        df = {}
        for document in self.documents_list:
            # word frequency in current document
            temp = {}
            # stat
            for word in document:
                temp[word] = temp.get(word, 0) + 1
            # save word frequency
            self.f.append(temp)
            # the number of documents containing the word key
            for key in temp.keys():
                df[key] = df.get(key, 0) + 1
        # compute word's idf
        for key, value in df.items():
            self.idf[key] = np.log((self.documents_number - value + 0.5) / (value + 0.5))

    # compute similarity score bewteen query and index-th document in all documents
    def get_score(self, index, query):
        score = 0.0
        document_len = len(self.documents_list[index])
        qf = Counter(query)
        for q in query:
            if q not in self.f[index]:
                continue
            score += self.idf[q] * (self.f[index][q] * (self.k1 + 1) / (
                        self.f[index][q] + self.k1 * (1 - self.b + self.b * document_len / self.avg_documents_len))) * (
                                 qf[q] * (self.k2 + 1) / (qf[q] + self.k2))

        return score
    # compute simialrity scores between query and all documents
    def get_documents_score(self, query):
        score_list = []
        for i in range(self.documents_number):
            score_list.append(self.get_score(i, query))
        return score_list

# Practice

Q1. Use the above BM25 model to compute the similary of a given query and documents

In [ ]:
query = "This is second documents"
corpus = [
    'this is the first document',
    'this is the second second document',
    'and the third one',
    'is this the first document'
]


# tokenize words
word_list = []
for i in range(len(corpus)):
    word_list.append(corpus[i].split(' '))

query = query.split(' ')

BM25 = BM25_Model(word_list)
scores = BM25.get_documents_score(query)

scores